# Sympy 2 -- Application to Newton's method

In this notebook we will focus on applying Sympy to accomplish a task. We will use it to set up Newton's method, comparing our initial point to the roots Newton's method (eventually) finds.

* * * 

Newton's method is the technique of iteratively using the linear approximation of a function to search for roots.  Say we want to find a solution to the equation

$$f(x) = 0$$

where, $f$ is a real-valued function of a real variable $f : \mathbb R \to \mathbb R$.

Start with an initial guess $x_0 \in \mathbb R$. The linear approximation $L$ at $x_0$ (to $f$) is the function

$$L_{x_0}(x) = f(x_0) + f'(x_0)(x-x_0).$$

Rather than trying to solve $f(x)=0$ we solve $L_{x_0}(x)=0$, which gives us

$$x = x_0-\frac{f(x_0)}{f'(x_0)}.$$

Newton's method is the process of replacing the initial guess $x_0$ with the solution to $L_{x_0}(x) = 0$, i.e. 

$$x_0 \longmapsto x_0-\frac{f(x_0)}{f'(x_0)}.$$

The function that sends $x_0$ to $x_0 -\frac{f(x_0)}{f'(x_0)}$ is called the **Newton iterate**, let's call this function $N_f$, i.e.

$$N_f(x) = x - \frac{f(x)}{f'(x)}.$$

* * *

For example, if $f(x) = x^2 - c$, then solving $f(x) = 0$ is equivalent to computing a square root of $c$.  On the other hand, the *Newton iterate* is the function

$$N_f(x) = x - \frac{x^2 - c}{2x} = \frac{x}{2} + \frac{c}{2x}.$$

This illustrates the compromise made with Newton's method: if one does not have a convenient direct way to compute square roots, one can get a good approximation by iteratively computing $N_f$, which uses only the operations of addition, multiplication and division. 

* * *

As an initial exercise, let's use Sympy to compute the Newton iterates of functions.  We will then make a plot of $f$, $N$ as well as applications of the Newton iterate to some initial guesses. 


In [118]:
import sympy as sp

